# Create Water Level Snapshot - pick time

**What this notebook does** This notebook creates a snapshot of latest water body conditions, appending the dam fill level as an attribute in a shapefile. This code needs to be run manually after new data has been ingested into DEA, and after the .txt files for each water bosy have been updated.

**Required inputs** 
- A shapefile to be appended with the latest data (`Shapefile_to_append`)
- A folder where all the `.txt files` are located from which the snapshot will be created (`Timeseries_dir`)
- The date for analysis needs to be updated below (`DateToExtractStr`)

**Date** December 2018

**Author** Claire Krause

In [1]:
import subprocess
import glob
import geopandas as gp
import re
import pandas as pd
from dateutil.relativedelta import relativedelta

## Set up the file paths, date to be extracted and output file names

`Timeseries_dir` is the folder where all the .txt files are located from which the snapshot will be created

`Shapefile_to_append` is the shapefile which the snapshot will be added to as an attribute

In [5]:
Timeseries_dir = '/g/data/r78/cek156/dea-notebooks/Dams/Dams2000to2018/Timeseries_bex/'
Shapefile_to_append = '/g/data/r78/rjd547/tmp/AllNSW2000to201810LatestSnapshot.shp'

SnapshotShp = gp.read_file(Shapefile_to_append)

### Choose when you want the snapshot for

Note that there is a filter built in here that excludes observations that are more than a month from the chosen date.

In [6]:
DateToExtractStr = '20181207'
DateToExtract = pd.to_datetime(DateToExtractStr, format='%Y%m%d')

# Create filter range which only includes observations of +/- 45 days
DateFilterA = DateToExtract + relativedelta(days=-45)
DateFilterB = DateToExtract + relativedelta(days=45)

## Get a list of all of the files in the folder to loop through

In [7]:
TimeSeriesFiles = glob.glob(f'{Timeseries_dir}*.csv')

## Loop through and extract the relevant date from all the .txt files

In [8]:
for file in TimeSeriesFiles:
    # Get the ID
    NameComponents = re.split('\.|/', file)  # Splitting on '.' and '/'
    PolyID = NameComponents[-2]
    PolyID = int(PolyID)
    try:
        AllObs = pd.read_csv(file, parse_dates=[
                             'Observation Date'], index_col='Observation Date')
        x = AllObs.iloc[AllObs.index.get_loc(DateToExtract, method='nearest')]
        if(x.name > DateFilterA and x.name < DateFilterB):
            ObsDate = str(x.name)
            Pct = float(x['Wet pixel percentage'])
            FindPolyIndex = SnapshotShp.where(
                SnapshotShp['ID'] == PolyID).dropna(how='all').index.values[0]
            SnapshotShp.loc[(FindPolyIndex, f'{DateToExtractStr}')] = ObsDate
            SnapshotShp.loc[(FindPolyIndex, f'Pct{DateToExtractStr}')] = Pct
        else:
            print(x.name, 'is out of snapshot range.')
    except:
        print(f'Bad {PolyID}')
        continue

Bad 40551
Bad 24671
2018-10-12 00:24:38 is out of snapshot range.
Bad 26402
Bad 53340
Bad 457
Bad 21171
Bad 38730
Bad 62961
Bad 29202
Bad 20848
Bad 43036
Bad 59221
Bad 11600
Bad 14751
Bad 11049
Bad 29175
Bad 44337
2018-08-23 23:42:30 is out of snapshot range.
Bad 59057
Bad 34484
Bad 7515
Bad 36020
Bad 46328
Bad 33969
Bad 1061
Bad 54204
Bad 29491
Bad 4935
Bad 24692
Bad 39115
Bad 33423
Bad 55638
Bad 28230
Bad 46631
Bad 21412
Bad 47507
Bad 58543
Bad 23379
Bad 16960
Bad 51637
Bad 51205
Bad 27554
Bad 47483
Bad 9781
Bad 15865
Bad 60581
Bad 17812
Bad 44577
Bad 55437
Bad 40154
Bad 41720
Bad 40704
Bad 2504
Bad 5046
Bad 21469
Bad 46298
Bad 20435
Bad 8488
Bad 46498
Bad 50641
Bad 15508
Bad 22034
Bad 49325
Bad 23342
Bad 6342
Bad 24847
Bad 54437
Bad 10434
Bad 7082
Bad 55815
Bad 36153
Bad 15754
Bad 7020
Bad 45723
Bad 47316
Bad 60001
Bad 32753
Bad 58566
Bad 14063
Bad 21332
Bad 45206
Bad 8771
Bad 22068
Bad 41490
Bad 39785
Bad 16249
Bad 29954
Bad 26336
Bad 9490
Bad 19601
Bad 8674
Bad 3199
Bad 27765
Bad 

Bad 59608
Bad 37909
Bad 20436
Bad 21283
Bad 49603
Bad 9321
Bad 54798
Bad 20348
Bad 11938
Bad 45197
Bad 43705
Bad 18975
Bad 2043
Bad 27905
Bad 18579
Bad 34193
Bad 62088
Bad 60525
Bad 11794
Bad 68
Bad 12645
Bad 45128
Bad 45462
Bad 9289
Bad 47390
Bad 47832
Bad 57772
Bad 12279
Bad 49841
Bad 2950
Bad 7274
Bad 9406
Bad 41394
Bad 33701
Bad 53826
Bad 28084
Bad 19793
Bad 45497
Bad 25084
Bad 49490
Bad 29632
Bad 16670
Bad 9896
Bad 23547
Bad 12678
Bad 44550
Bad 53216
Bad 61421
Bad 9278
Bad 61827
Bad 21338
Bad 63790
Bad 36826
Bad 26111
Bad 50222
Bad 24448
Bad 7978
Bad 59467
Bad 35079
Bad 38837
Bad 26014
Bad 44554
Bad 231
Bad 50860
Bad 55344
Bad 54103
Bad 30903
Bad 53132
Bad 40331
Bad 14177
Bad 44806
Bad 43639
Bad 34053
Bad 53532
Bad 12265
Bad 39618
Bad 31933
Bad 3164
Bad 42677
Bad 8508
Bad 4873
Bad 46102
Bad 35593
Bad 41558
Bad 46842
Bad 22660
Bad 60069
Bad 43713
Bad 9846
Bad 13560
Bad 18995
Bad 37541
Bad 33258
Bad 59497
Bad 39697
Bad 21354
Bad 5797
Bad 47749
Bad 21016
Bad 30882
Bad 46089
Bad 4708


Bad 5946
Bad 46802
Bad 28617
Bad 31388
Bad 31722
Bad 8655
Bad 63391
Bad 50239
Bad 2120
Bad 29966
Bad 30567
Bad 17430
Bad 50142
Bad 34246
Bad 34710
Bad 11388
Bad 34560
Bad 8105
Bad 62208
Bad 61864
Bad 45727
2011-11-16 23:31:56 is out of snapshot range.
Bad 31485
Bad 30514
Bad 50445
Bad 4550
Bad 55607
Bad 56147
Bad 33526
Bad 9872
Bad 41804
Bad 49594
Bad 48328
Bad 22257
Bad 19705
Bad 51426
Bad 3340
Bad 49521
Bad 21444
Bad 50345
Bad 30159
Bad 51361
Bad 45813
Bad 42273
Bad 5500
Bad 51694
Bad 48486
Bad 29532
Bad 31781
Bad 75
Bad 749
Bad 46476
Bad 27334
Bad 24625
Bad 14276
Bad 8953
Bad 50592
Bad 10457
Bad 61835
Bad 54659
Bad 49571
Bad 45310
Bad 2049
Bad 2990
Bad 5015
Bad 59312
Bad 37211
Bad 33011
Bad 46116
Bad 45222
Bad 21904
Bad 53257
2008-02-09 23:35:11 is out of snapshot range.
Bad 13938
Bad 28924
Bad 59272
Bad 31946
Bad 509
Bad 163
Bad 31848
Bad 1085
Bad 52152
Bad 1007
Bad 1234
Bad 29867
Bad 29693
Bad 52466
2011-11-16 23:31:56 is out of snapshot range.
Bad 40357
Bad 18555
Bad 12618
Bad 63

Bad 59517
Bad 37971
Bad 61766
Bad 45184
Bad 52168
Bad 43584
Bad 24000
Bad 62324
Bad 17747
Bad 10418
Bad 58688
Bad 19702
Bad 53834
Bad 48746
Bad 34697
Bad 41073
Bad 58348
Bad 56860
Bad 17234
Bad 4702
Bad 1220
Bad 16641
Bad 42706
Bad 23353
Bad 46152
Bad 45659
Bad 37666
Bad 30216
Bad 40736
Bad 25398
2018-10-04 00:27:08 is out of snapshot range.
Bad 46949
Bad 21129
Bad 21577
Bad 5153
Bad 63285
Bad 25049
Bad 46274
Bad 36059
Bad 19143
Bad 34974
Bad 61163
Bad 34865
Bad 42985
Bad 31565
Bad 22916
Bad 1468
Bad 18902
Bad 31099
Bad 60751
Bad 49856
Bad 58576
Bad 49082
Bad 18494
Bad 60448
Bad 33110
Bad 48467
Bad 38302
Bad 42000
Bad 46313
Bad 23873
Bad 48583
Bad 32238
Bad 51835
Bad 13621
Bad 12932
Bad 46381
Bad 34044
Bad 22267
Bad 45085
Bad 2265
Bad 6397
Bad 905
Bad 21879
Bad 50165
Bad 19445
Bad 29462
Bad 12442
Bad 59993
Bad 50086
Bad 50190
Bad 29731
Bad 58509
Bad 2635
Bad 26828
Bad 3747
Bad 24913
Bad 52838
Bad 63191
Bad 28485
Bad 15254
Bad 43124
Bad 48061
Bad 33628
Bad 58520
Bad 33574
Bad 9473
Bad 4

Bad 51281
Bad 26371
Bad 41208
Bad 30161
Bad 45248
Bad 11667
Bad 53249
Bad 30038
Bad 540
Bad 47350
Bad 36223
Bad 3913
Bad 37992
Bad 52244
Bad 61261
Bad 20188
Bad 15799
Bad 44604
Bad 21077
Bad 34686
Bad 61003
Bad 21059
Bad 28410
Bad 50050
Bad 19334
Bad 56815
Bad 3394
Bad 57113
Bad 25515
Bad 24749
Bad 20851
Bad 5835
Bad 32985
Bad 1326
Bad 51021
Bad 2518
2018-10-12 00:24:38 is out of snapshot range.
Bad 49613
Bad 28373
Bad 54169
Bad 31259
Bad 41071
Bad 51542
Bad 3357
Bad 23177
Bad 2627
Bad 43111
Bad 53376
Bad 31463
Bad 8765
Bad 18980
Bad 42167
Bad 29713
Bad 62
Bad 5425
Bad 30104
Bad 12343
Bad 5081
Bad 39049
Bad 57399
Bad 40445
Bad 29450
Bad 55252
Bad 42862
Bad 11896
Bad 39067
Bad 40464
Bad 24736
Bad 47515
Bad 11760
Bad 7846
Bad 18307
Bad 60866
Bad 47688
Bad 21320
Bad 60922
Bad 36650
Bad 22284
Bad 45784
Bad 47376
Bad 46612
Bad 59801
Bad 42798
Bad 52006
Bad 52058
Bad 58930
Bad 28730
Bad 30103
Bad 41379
Bad 37429
Bad 35776
Bad 41761
Bad 56897
Bad 20828
Bad 12665
Bad 31986
Bad 32627
Bad 8108
B

...

Bad 46999
Bad 6066
Bad 9226
Bad 3291
Bad 51872
Bad 374
Bad 52362
Bad 42793
Bad 46524
Bad 15740
Bad 37946
Bad 49241
Bad 38060
Bad 22815
Bad 63048
Bad 27809
Bad 17682
Bad 44837
Bad 24550
Bad 11123
Bad 57028
Bad 60675
Bad 47212
Bad 23814
Bad 28689
Bad 47188
Bad 16848
Bad 62880
Bad 50883
Bad 25619
Bad 58478
Bad 3296
Bad 34525
Bad 3045
Bad 33233
Bad 7390
Bad 22910
2018-08-23 23:42:30 is out of snapshot range.
Bad 57968
Bad 14098
Bad 55575
Bad 21192
Bad 22095
Bad 11329
Bad 28667
Bad 36783
Bad 32787
Bad 38074
Bad 18367
Bad 8259
Bad 29608
Bad 33284
Bad 18802
Bad 14014
Bad 3299
Bad 50949
Bad 19734
Bad 42601
Bad 52975
Bad 32743
Bad 54818
Bad 8763
Bad 10331
Bad 46662
Bad 31957
Bad 11948
Bad 897
Bad 9361
Bad 12637
Bad 24083
Bad 12647
Bad 41290
Bad 44171
Bad 11974
Bad 5531
Bad 44987
Bad 41740
Bad 61841
Bad 17144
Bad 45223
Bad 50335
Bad 15021
Bad 58290
Bad 25260
Bad 39905
Bad 46448
Bad 44005
Bad 29899
Bad 57660
Bad 38585
Bad 57451
Bad 57018
Bad 56639
Bad 63635
Bad 31316
Bad 60081
Bad 35145
Bad 10555

Bad 7300
Bad 46592
Bad 51720
Bad 46906
Bad 38938
Bad 28965
Bad 20159
Bad 22402
Bad 52691
Bad 832
Bad 34963
Bad 58979
Bad 12795
Bad 31468
Bad 36848
Bad 60537
Bad 57261
Bad 56230
Bad 36818
Bad 35814
Bad 26140
Bad 24041
Bad 31519
Bad 14399
Bad 52886
Bad 57008
Bad 15955
Bad 41248
Bad 34364
Bad 57838
Bad 54682
Bad 30722
Bad 42742
Bad 24490
Bad 51791
Bad 58285
Bad 4632
Bad 37357
Bad 11045
Bad 56094
Bad 46291
Bad 18423
Bad 54551
Bad 56002
Bad 11624
Bad 2136
Bad 39812
Bad 13861
Bad 13125
Bad 18841
Bad 5961
Bad 62072
Bad 5207
Bad 40434
Bad 47052
Bad 13190
Bad 16593
Bad 60062
Bad 52068
Bad 41888
Bad 31136
Bad 38121
Bad 26156
Bad 59133
Bad 49170
Bad 35561
Bad 25243
Bad 59751
Bad 16750
Bad 39028
Bad 1163
Bad 46242
Bad 50698
2008-01-15 23:42:12 is out of snapshot range.
Bad 42998
Bad 9436
Bad 31611
Bad 9435
Bad 20896
Bad 34217
Bad 8299
Bad 14388
Bad 62274
Bad 60645
Bad 29677
Bad 57736
Bad 6359
Bad 37526
Bad 35
Bad 7736
Bad 29873
Bad 5637
Bad 19854
Bad 28312
Bad 13966
Bad 10354
Bad 63007
Bad 34877
B

Bad 37665
Bad 39721
Bad 24156
Bad 29278
Bad 10157
Bad 60683
Bad 50496
Bad 27508
Bad 1652
Bad 48989
Bad 402
Bad 33219
Bad 16479
Bad 55959
Bad 54265
Bad 36380
Bad 27211
Bad 47121
Bad 32587
Bad 4198
Bad 21265
Bad 7624
Bad 62203
Bad 35576
Bad 30390
Bad 13210
Bad 38651
Bad 13483
Bad 18667
Bad 53264
Bad 42647
Bad 3076
Bad 36412
Bad 15015
Bad 48639
Bad 41088
Bad 36895
Bad 34423
Bad 20340
Bad 56151
Bad 57996
Bad 3178
Bad 41875
Bad 11341
Bad 1963
Bad 28882
Bad 42387
Bad 27362
Bad 24190
Bad 7962
Bad 7161
Bad 45270
Bad 45881
Bad 48660
Bad 61501
Bad 24765
Bad 2395
Bad 33445
Bad 48025
Bad 50534
Bad 30323
Bad 13181
Bad 2952
Bad 997
Bad 49633
Bad 29916
Bad 21782
2008-02-09 23:34:47 is out of snapshot range.
Bad 42088
Bad 41800
Bad 38021
Bad 3184
Bad 36519
Bad 34961
Bad 18945
Bad 51901
Bad 9279
Bad 18209
Bad 3578
Bad 44042
Bad 31764
Bad 20527
Bad 25687
Bad 52399
Bad 18823
Bad 57989
Bad 703
Bad 30575
Bad 58526
Bad 40801
Bad 30291
Bad 46608
Bad 21498
Bad 29160
Bad 27900
Bad 27966
Bad 30693
Bad 44956
Bad

Bad 61844
Bad 60962
Bad 37142
Bad 44610
Bad 32796
Bad 5458
Bad 58624
Bad 1283
Bad 21001
Bad 61637
Bad 10782
Bad 16418
Bad 34092
Bad 7849
Bad 62214
Bad 58241
Bad 48148
Bad 41717
Bad 46913
Bad 47472
Bad 16192
Bad 39018
Bad 31208
Bad 12848
Bad 35610
Bad 47411
Bad 58039
Bad 26925
Bad 62295
Bad 49106
Bad 9400
Bad 21610
Bad 45062
Bad 32114
Bad 48623
Bad 15682
Bad 63593
Bad 42291
Bad 41890
Bad 26556
Bad 32289
2008-02-09 23:35:35 is out of snapshot range.
Bad 8521
Bad 40878
Bad 34519
Bad 29267
Bad 13391
Bad 14429
Bad 57492
Bad 17568
Bad 57515
Bad 33857
Bad 30124
Bad 16329
Bad 24689
Bad 59955
Bad 23485
Bad 62211
Bad 51091
Bad 50958
Bad 37064
Bad 43162
Bad 11410
Bad 20635
Bad 19008
Bad 9154
Bad 22197
Bad 62991
Bad 63453
Bad 458
Bad 34151
Bad 43169
Bad 38230
Bad 9295
Bad 30114
Bad 7610
Bad 22075
2008-02-09 23:34:47 is out of snapshot range.
Bad 4270
Bad 30373
Bad 41028
Bad 49642
Bad 55273
Bad 39701
Bad 43646
2007-09-02 23:37:42 is out of snapshot range.
Bad 3270
Bad 22509
Bad 21783
Bad 61353
Bad 

Bad 63017
Bad 18944
Bad 52057
Bad 25367
Bad 45363
Bad 56627
Bad 52163
Bad 11215
Bad 43094
Bad 24901
2008-02-09 23:34:47 is out of snapshot range.
Bad 14040
Bad 22300
Bad 32030
Bad 19959
Bad 3038
Bad 28675
Bad 6013
Bad 12561
Bad 56799
Bad 18779
Bad 13555
Bad 33431
Bad 21639
Bad 45901
Bad 51016
Bad 36869
Bad 35543
Bad 23451
Bad 30254
Bad 42641
Bad 53480
Bad 48541
Bad 50798
Bad 3253
Bad 51278
Bad 44261
Bad 12141
Bad 16794
Bad 56739
Bad 28975
Bad 63531
Bad 43362
Bad 45848
Bad 47756
Bad 11844
Bad 48284
Bad 1676
Bad 14005
Bad 9581
Bad 15444
Bad 52714
Bad 52926
Bad 14348
Bad 1892
Bad 34256
Bad 12116
Bad 53688
Bad 28706
Bad 769
Bad 61719
Bad 22571
Bad 10658
Bad 17380
Bad 29475
Bad 16693
Bad 59096
Bad 62031
Bad 23103
Bad 35653
Bad 24429
Bad 5907
Bad 15812
Bad 25383
Bad 52073
Bad 24950
Bad 3031
Bad 38828
Bad 61134
Bad 21066
Bad 1233
Bad 17721
Bad 38777
Bad 5626
2008-02-09 23:34:23 is out of snapshot range.
Bad 44992
Bad 9126
Bad 11296
Bad 7227
Bad 24107
Bad 27681
Bad 22423
Bad 1328
Bad 18214
Bad

## Write out the appended shapefile

In [6]:
schema = gp.io.file.infer_schema(SnapshotShp)
schema['properties'][f'{DateToExtractStr}'] = 'str'
schema['properties'][f'Pct{DateToExtractStr}'] = 'float' 
SnapshotShp.to_file(Shapefile_to_append, schema = schema)

In [ ]:
print(schema['properties'][f'{DateToExtractStr}'])
print(schema)